In [5]:
!brew install fluidsynth
!pip install pyfluidsynth


==> Fetching downloads for: fluid-synth
==> Downloading https://ghcr.io/v2/homebrew/core/fluid-synth/manifests/2.5.1
######################################################################### 100.0%
==> Fetching fluid-synth
==> Downloading https://ghcr.io/v2/homebrew/core/fluid-synth/blobs/sha256:f1de36
######################################################################### 100.0%
==> Pouring fluid-synth--2.5.1.arm64_tahoe.bottle.tar.gz
🍺  /opt/homebrew/Cellar/fluid-synth/2.5.1: 43 files, 3.2MB
==> Running `brew cleanup fluid-synth`...
Disable this behaviour by setting `HOMEBREW_NO_INSTALL_CLEANUP=1`.
Hide these hints with `HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).


In [1]:
import pretty_midi
import numpy as np
import pretty_midi
from pythreejs import *
from IPython.display import display
import time

midi_data = pretty_midi.PrettyMIDI("/Users/ajanshul02gmail.com/T-5/Project/Data sets/new.mid")
pm = pretty_midi.PrettyMIDI("/Users/ajanshul02gmail.com/T-5/Project/Data sets/new.mid")
audio_data = pm.fluidsynth()  # convert to audio (optional, requires fluidsynth)
notes = []
for inst in pm.instruments:
    for note in inst.notes:
        notes.append((note.start, note.end, note.pitch, note.velocity))
notes = sorted(notes, key=lambda x: x[0])

# Setup scene
camera = PerspectiveCamera(position=[0, 0, 5])
scene = Scene(children=[
    AmbientLight(color='#777'),
    DirectionalLight(color='white', position=[3, 5, 1]),
])
geometry = BufferGeometry(attributes={'position': BufferAttribute(np.random.normal(size=(2000,3)).astype('float32'))})
material = PointsMaterial(color='white', size=0.05)
points = Points(geometry=geometry, material=material)
scene.add(points)
renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)],
                    width=600, height=400)
display(renderer)

# Animation loop based on MIDI timing
start = time.time()
i = 0
while i < len(notes):
    now = time.time() - start
    start_t, end_t, pitch, vel = notes[i]
    if now >= start_t:
        hue = (pitch - 20) / 100  # map pitch to color
        material.color = f"hsl({hue*360},100%,50%)"
        scale = 1 + vel / 127 * 5
        points.scale = (scale, scale, scale)
        i += 1
    time.sleep(0.01)


Renderer(camera=PerspectiveCamera(position=(0.0, 0.0, 5.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.…

In [5]:
import io
import asyncio
import pretty_midi
import numpy as np
from pythreejs import *
from IPython.display import display
import ipywidgets as widgets

# -----------------------------
# UI widgets
# -----------------------------
upload = widgets.FileUpload(
    accept='.mid,.midi',
    multiple=False,
    description='🎵 Add MIDI File'
)

play_button = widgets.Button(
    description='▶️ Play Animation',
    button_style='success',
    disabled=True
)

output = widgets.Output()

display(widgets.VBox([upload, play_button, output]))

# -----------------------------
# 3D Scene setup
# -----------------------------
camera = PerspectiveCamera(position=[0, 0, 5])
scene = Scene(children=[
    AmbientLight(color='#777'),
    DirectionalLight(color='white', position=[3, 5, 1]),
])

geometry = BufferGeometry(attributes={
    'position': BufferAttribute(np.random.normal(size=(2000, 3)).astype('float32'))
})
material = PointsMaterial(color='white', size=0.05)
points = Points(geometry=geometry, material=material)
scene.add(points)

renderer = Renderer(
    camera=camera,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=700, height=450
)

display(renderer)

# -----------------------------
# Helper: Parse uploaded MIDI
# -----------------------------
midi_data = None
notes = []

def parse_midi(file_content):
    global midi_data, notes
    midi_data = pretty_midi.PrettyMIDI(io.BytesIO(file_content))
    notes = []
    for inst in midi_data.instruments:
        for note in inst.notes:
            notes.append((note.start, note.end, note.pitch, note.velocity))
    notes.sort(key=lambda x: x[0])

# -----------------------------
# Upload handler
# -----------------------------
def on_upload_change(change):
    if not upload.value:
        return

    # Handle both tuple and dict formats
    value = upload.value
    if isinstance(value, dict):
        file_info = next(iter(value.values()))
        content = file_info['content']
        name = file_info['metadata']['name']
    elif isinstance(value, tuple):
        file_info = value[0]
        content = file_info['content']
        name = file_info['name']
    else:
        raise ValueError(f"Unexpected upload.value format: {type(value)}")

    # Parse MIDI and enable play button
    parse_midi(content)
    play_button.disabled = False

    with output:
        output.clear_output()
        print(f"✅ Loaded MIDI file: {name}")
upload.observe(on_upload_change, names='value')

# -----------------------------
# Async animation function
# -----------------------------
async def animate():
    start = asyncio.get_event_loop().time()
    i = 0
    while i < len(notes):
        now = asyncio.get_event_loop().time() - start
        start_t, end_t, pitch, vel = notes[i]
        if now >= start_t:
            hue = (pitch - 20) / 100  # map pitch to color
            material.color = f"hsl({hue*360},100%,50%)"
            scale = 1 + vel / 127 * 5
            points.scale = (scale, scale, scale)
            i += 1
        await asyncio.sleep(0.01)

# -----------------------------
# Play button handler
# -----------------------------
def on_play_clicked(b):
    if notes:
        with output:
            output.clear_output()
            print("🎬 Playing animation...")
        asyncio.create_task(animate())
    else:
        with output:
            output.clear_output()
            print("⚠️ Please upload a MIDI file first!")

play_button.on_click(on_play_clicked)


Renderer(camera=PerspectiveCamera(position=(0.0, 0.0, 5.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.…

In [4]:
pip install fluidsynth  

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fluidsynth: filename=fluidsynth-0.2-py3-none-any.whl size=4513 sha256=79ff5dfa037736b13ad92921242985c488eeabd4227023923844119155e666c3
  Stored in directory: /Users/ajanshul02gmail.com/Library/Caches/pip/wheels/b2/b2/ec/2f935ba979ac99f4fd20690fcb5be074859d2ab1ebeb3c0ae4
Successfully built fluidsynth
Note: you may need to restart the kernel to use updated packages.


In [1]:
import io
import asyncio
import numpy as np
import pretty_midi
import ipywidgets as widgets
from pythreejs import *
from IPython.display import display, Audio
import pyfluidsynth

# --- UI ---
upload = widgets.FileUpload(accept='.mid,.midi', multiple=False, description='🎵 Add MIDI File')
play_button = widgets.Button(description='▶️ Play with Sound', button_style='success', disabled=True)
output = widgets.Output()

display(widgets.VBox([upload, play_button, output]))

# --- Scene setup ---
camera = PerspectiveCamera(position=[0, 0, 10])
ambient = AmbientLight(color='#555')
light = DirectionalLight(color='white', position=[3, 5, 1])
scene = Scene(children=[ambient, light])

particles = []
renderer = Renderer(
    camera=camera,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=800, height=500
)
display(renderer)

# --- Global state ---
notes = []
audio_widget = None

# --- Load and parse MIDI ---
def parse_midi(file_content):
    global notes
    midi = pretty_midi.PrettyMIDI(io.BytesIO(file_content))
    notes = []
    for inst in midi.instruments:
        for note in inst.notes:
            notes.append((note.start, note.end, note.pitch, note.velocity))
    notes.sort(key=lambda x: x[0])
    return midi

def on_upload_change(change):
    global midi_data
    if not upload.value:
        return
    value = upload.value
    if isinstance(value, dict):
        file_info = next(iter(value.values()))
        content = file_info['content']
        name = file_info['metadata']['name']
    elif isinstance(value, tuple):
        file_info = value[0]
        content = file_info['content']
        name = file_info['name']
    else:
        raise ValueError(f"Unexpected upload.value format: {type(value)}")

    midi_data = parse_midi(content)
    play_button.disabled = False
    with output:
        output.clear_output()
        print(f"✅ Loaded: {name} — {len(notes)} notes found")

upload.observe(on_upload_change, names='value')

# --- Play MIDI sound and animation ---
async def animate_with_sound(midi):
    global audio_widget
    with output:
        output.clear_output()
        print("🎬 Playing MIDI with sound and animation...")

    # Create audio
    fs = pyfluidsynth.Synth()
    sfid = fs.sfload("/System/Library/Components/DLSMusicDevice.component/Contents/Resources/gs_instruments.dls")  # macOS soundfont
    fs.program_select(0, sfid, 0, 0)
    audio = midi.fluidsynth(fs=fs)
    fs.delete()

    # Display audio player
    audio_widget = Audio(audio, rate=44100, autoplay=True)
    display(audio_widget)

    # Animation
    start_time = asyncio.get_event_loop().time()
    active = []
    while True:
        now = asyncio.get_event_loop().time() - start_time

        # Add notes that start now
        for (s, e, p, v) in [n for n in notes if s <= now < s + 0.05]:
            hue = (p - 20) / 80
            color = f"hsl({hue * 360},100%,60%)"
            size = 0.2 + (v / 127) * 0.6
            pos = [np.random.uniform(-5, 5), np.random.uniform(-3, 3), np.random.uniform(-1, 1)]
            geom = SphereGeometry(radius=size)
            mat = MeshStandardMaterial(color=color, emissive=color)
            particle = Mesh(geom, mat, position=pos)
            scene.add(particle)
            active.append((particle, now, e))

        # Fade out particles
        for (particle, start, end) in active[:]:
            age = now - start
            if now > end:
                scene.remove(particle)
                active.remove((particle, start, end))
            else:
                particle.scale = [1 - 0.5 * (age / (end - start))] * 3

        await asyncio.sleep(0.01)

        if now > notes[-1][1]:
            break

# --- Play button ---
def on_play_clicked(b):
    asyncio.create_task(animate_with_sound(midi_data))

play_button.on_click(on_play_clicked)


ModuleNotFoundError: No module named 'pyfluidsynth'